In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ast import literal_eval
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import MeanSquaredError, Accuracy
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Concatenate, Input, Dropout,  LSTM, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.regularizers import l1_l2
from keras.models import Sequential
from transformers import TFAutoModel, AutoTokenizer


/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('/Users/louishagenbucher/Documents/GitHub/mtg_nlp_price_prediction/data/clean_cards.csv')
df.sample(5)

,cmc,colors,legendary,types,sub_types,oracle_text,keywords,power,toughness,rarity,reserved,released_at,booster,usd
15700,3,['W'],False,['Sorcery'],['none'],Return up to three target creature cards with ...,['no keywords'],0,0,uncommon,False,10376,True,0.04
13141,2,['W'],False,['Creature'],['Cat'],Vigilance,['Vigilance'],1,4,common,False,9641,True,0.02
4806,7,"['B', 'G', 'R']",True,['Creature'],['Dragon'],"Flying, trample, haste\nIf Darigaaz Reincarnat...","['Flying', 'Haste', 'Trample']",7,7,mythic,False,8913,True,0.54
13539,6,"['G', 'U']",False,['Creature'],"['Merfolk', 'Soldier']",Flash (You may cast this spell any time you co...,['Flash'],4,2,uncommon,False,7002,True,0.06
3475,5,['G'],False,['Creature'],['Shapeshifter'],Changeling (This card is every creature type.)...,"['Champion', 'Changeling']",7,7,uncommon,False,5063,True,0.68


In [4]:
no_prices_df = df[df['usd'].isna()]

In [5]:
df.drop(df[df['usd'].isna()].index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [6]:
df['colors'] = df['colors'].apply(literal_eval)

In [7]:
mlb = MultiLabelBinarizer()
df= df.join(pd.DataFrame(mlb.fit_transform(df['colors']),
                                      columns=mlb.classes_,
                                      index=df.index))
legend_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['legendary']), columns=['legendary'])
             .add_prefix('_'))
booster_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['booster']), columns=['booster'])
             .add_prefix('_'))
reserved_lb= LabelBinarizer()
df = df.join(pd.DataFrame(legend_lb.fit_transform(df['reserved']), columns=['resrved'])
             .add_prefix('_'))

df = df.drop(['legendary', 'booster', 'reserved', 'colors'], axis=1)
df.head(3)

,cmc,types,sub_types,oracle_text,keywords,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved
0,2,['Artifact'],['Equipment'],Equipped creature gets +2/+2.\nEquip {3} ({3}:...,['Equip'],0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0
1,4,['Creature'],"['Bird', 'Rogue']",Flying\nWhen Aarakocra Sneak enters the battle...,['Flying'],1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0
2,5,['Creature'],"['Astartes', 'Warrior']",Trample\nMark of Chaos Ascendant — During your...,"['Mark of Chaos Ascendant', 'Trample']",5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0


#### Text processing

In [8]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = TFAutoModel.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [9]:
df['text_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['oracle_text']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = np.split(last_hidden_states, len(batch_df))

    for j, embedding in enumerate(batch_embeddings):
        df.at[i + j, 'text_embedding'] = embedding.tolist()

df['text_embedding'] = df['text_embedding'].tolist()


: 

In [ ]:
df['sub_types_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['sub_types']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state.numpy()

    batch_embeddings = np.split(last_hidden_states, len(batch_df))

    for j, embedding in enumerate(batch_embeddings):
        df.at[i + j, 'sub_types_embedding'] = embedding.tolist()

df['sub_types_embedding'] = df['sub_types_embedding'].tolist()


In [ ]:
df['types_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['types']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'types_embedding'] = batch_embeddings

df['types_embedding'] = df['types_embedding'].tolist()

In [ ]:
df['keywords_embedding'] = None

for i in range(0, len(df), 32):
    batch_df = df.iloc[i:i+32]

    batch_tokens = tokenizer(list(batch_df['keywords']), return_tensors='tf', padding=True, truncation=True)

    outputs = model(**batch_tokens)
    last_hidden_states = outputs.last_hidden_state

    batch_embeddings = tf.split(last_hidden_states, num_or_size_splits=len(batch_df))
    df.loc[i:i+32-1, 'keywords_embedding'] = batch_embeddings

df['keywords_embedding'] = df['keywords_embedding'].tolist()

In [ ]:
df.columns

Index(['cmc', 'types', 'sub_types', 'oracle_text', 'keywords', 'power',
       'toughness', 'rarity', 'released_at', 'usd', 'B', 'C', 'G', 'N', 'R',
       'U', 'W', '_legendary', '_booster', '_resrved', 'sub_types_embedding',
       'types_embedding', 'keywords_embedding', 'text_embedding'],
      dtype='object')

In [ ]:
df.drop(columns=['sub_types','types', 'keywords', 'oracle_text'],  inplace=True)

#### The Pipeline and neural network

In [ ]:
df

,cmc,power,toughness,rarity,released_at,usd,B,C,G,N,R,U,W,_legendary,_booster,_resrved,sub_types_embedding,types_embedding,keywords_embedding,text_embedding
0,2,0,0,common,10096,0.02,0,0,0,0,0,0,1,0,1,0,"(((tf.Tensor(0.048333786, shape=(), dtype=floa...","(((tf.Tensor(-0.055276375, shape=(), dtype=flo...","(((tf.Tensor(-0.039147772, shape=(), dtype=flo...","(((tf.Tensor(-0.36550942, shape=(), dtype=floa..."
1,4,1,4,common,10418,0.06,0,0,0,0,0,1,0,0,1,0,"(((tf.Tensor(-0.007576871, shape=(), dtype=flo...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(-0.011417619, shape=(), dtype=flo...","(((tf.Tensor(-0.166848, shape=(), dtype=float3..."
2,5,5,5,mythic,10537,2.81,1,0,0,0,1,1,0,1,0,0,"(((tf.Tensor(-0.052856985, shape=(), dtype=flo...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(0.08564389, shape=(), dtype=float...","(((tf.Tensor(-0.23658565, shape=(), dtype=floa..."
3,0,0,0,common,2861,0.14,0,0,0,1,0,0,0,0,1,0,"(((tf.Tensor(-0.03981906, shape=(), dtype=floa...","(((tf.Tensor(0.038633786, shape=(), dtype=floa...","(((tf.Tensor(-0.046696614, shape=(), dtype=flo...","(((tf.Tensor(-0.36838064, shape=(), dtype=floa..."
4,3,0,0,rare,9634,0.09,0,1,0,0,0,0,0,0,0,0,"(((tf.Tensor(-0.03981906, shape=(), dtype=floa...","(((tf.Tensor(-0.055276375, shape=(), dtype=flo...","(((tf.Tensor(-0.046696614, shape=(), dtype=flo...","(((tf.Tensor(-0.037254475, shape=(), dtype=flo..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25484,5,7,2,mythic,7604,0.42,1,0,0,0,1,0,1,1,1,0,"(((tf.Tensor(-0.12533769, shape=(), dtype=floa...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(-0.0920035, shape=(), dtype=float...","(((tf.Tensor(-0.22740552, shape=(), dtype=floa..."
25485,4,0,0,rare,4258,0.82,0,0,0,0,0,1,0,0,1,0,"(((tf.Tensor(-0.03981906, shape=(), dtype=floa...","(((tf.Tensor(0.00986504, shape=(), dtype=float...","(((tf.Tensor(-0.04669656, shape=(), dtype=floa...","(((tf.Tensor(-0.33752233, shape=(), dtype=floa..."
25486,4,1,4,rare,10446,0.53,1,0,0,0,0,1,1,1,1,0,"(((tf.Tensor(-0.022519136, shape=(), dtype=flo...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(-0.011417619, shape=(), dtype=flo...","(((tf.Tensor(-0.22195312, shape=(), dtype=floa..."
25487,3,2,3,rare,10376,0.25,0,0,0,0,1,0,0,1,0,0,"(((tf.Tensor(-0.02203759, shape=(), dtype=floa...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(-0.04669656, shape=(), dtype=floa...","(((tf.Tensor(-0.3002531, shape=(), dtype=float..."


In [ ]:
X = df.drop(columns='usd')
y = df['usd']

In [ ]:
numerical_features = ['power', 'toughness', 'released_at', 'cmc']
categorical_features = ['rarity']
remaining_features = X.drop(columns=['rarity', 'power', 'toughness', 'released_at', 'cmc'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ],
    remainder='passthrough'
)

pipeline = make_pipeline(preprocessor)

In [ ]:
X_preprocessed= pipeline.fit_transform(X)

In [ ]:
column_names = pipeline.get_feature_names_out()
column_names

array(['num__power', 'num__toughness', 'num__released_at', 'num__cmc',
       'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare',
       'cat__rarity_special', 'cat__rarity_uncommon', 'remainder__B',
       'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R',
       'remainder__U', 'remainder__W', 'remainder___legendary',
       'remainder___booster', 'remainder___resrved',
       'remainder__sub_types_embedding', 'remainder__types_embedding',
       'remainder__keywords_embedding', 'remainder__text_embedding'],
      dtype=object)

In [ ]:
processed_df = pd.DataFrame(X_preprocessed, columns=column_names)

In [ ]:
processed_df.head(3)

,num__power,num__toughness,num__released_at,num__cmc,cat__rarity_common,cat__rarity_mythic,cat__rarity_rare,cat__rarity_special,cat__rarity_uncommon,remainder__B,...,remainder__R,remainder__U,remainder__W,remainder___legendary,remainder___booster,remainder___resrved,remainder__sub_types_embedding,remainder__types_embedding,remainder__keywords_embedding,remainder__text_embedding
0,-0.787786,-0.818064,0.83164,-0.74473,1.0,0.0,0.0,0.0,0.0,0,...,0,0,1,0,1,0,"(((tf.Tensor(0.048333786, shape=(), dtype=floa...","(((tf.Tensor(-0.055276375, shape=(), dtype=flo...","(((tf.Tensor(-0.039147772, shape=(), dtype=flo...","(((tf.Tensor(-0.36550942, shape=(), dtype=floa..."
1,-0.237532,1.302998,0.930864,0.397826,1.0,0.0,0.0,0.0,0.0,0,...,0,1,0,0,1,0,"(((tf.Tensor(-0.007576871, shape=(), dtype=flo...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(-0.011417619, shape=(), dtype=flo...","(((tf.Tensor(-0.166848, shape=(), dtype=float3..."
2,1.963486,1.833263,0.967534,0.969104,0.0,1.0,0.0,0.0,0.0,1,...,1,1,0,1,0,0,"(((tf.Tensor(-0.052856985, shape=(), dtype=flo...","(((tf.Tensor(-0.004934013, shape=(), dtype=flo...","(((tf.Tensor(0.08564389, shape=(), dtype=float...","(((tf.Tensor(-0.23658565, shape=(), dtype=floa..."


In [ ]:
processed_df =processed_df.astype({
    'num__power': float,
    'num__toughness': float,
    'num__released_at': float,
    'num__cmc': float
})

In [ ]:
processed_df = processed_df.astype({
    "cat__rarity_common": int,
    "cat__rarity_mythic": int,
    "cat__rarity_rare": int,
    "cat__rarity_special": int,
    "cat__rarity_uncommon": int,
    "remainder__B": int,
    "remainder__C": int,
    "remainder__G": int,
    "remainder__N": int,
    "remainder__R": int,
    "remainder__U": int,
    "remainder__W": int,
    "remainder___legendary": int,
    "remainder___booster": int,
    "remainder___resrved": int
})

In [ ]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25489 entries, 0 to 25488
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   num__power                      25489 non-null  float64
 1   num__toughness                  25489 non-null  float64
 2   num__released_at                25489 non-null  float64
 3   num__cmc                        25489 non-null  float64
 4   cat__rarity_common              25489 non-null  int64  
 5   cat__rarity_mythic              25489 non-null  int64  
 6   cat__rarity_rare                25489 non-null  int64  
 7   cat__rarity_special             25489 non-null  int64  
 8   cat__rarity_uncommon            25489 non-null  int64  
 9   remainder__B                    25489 non-null  int64  
 10  remainder__C                    25489 non-null  int64  
 11  remainder__G                    25489 non-null  int64  
 12  remainder__N                    

In [ ]:
X_numerical = processed_df[['num__power', 'num__toughness', 'num__released_at', 'num__cmc']]
X_categorical = processed_df[['remainder___legendary', 'remainder___booster', 'remainder___resrved',
                              'cat__rarity_common', 'cat__rarity_mythic', 'cat__rarity_rare', 'cat__rarity_special', 'cat__rarity_uncommon',
                              'remainder__B', 'remainder__C', 'remainder__G', 'remainder__N', 'remainder__R', 'remainder__U', 'remainder__W']]
X_text = processed_df[['remainder__text_embedding']]
X_subtypes= processed_df[['remainder__sub_types_embedding']]
X_types = processed_df[['remainder__types_embedding']]
X_keys = processed_df[['remainder__keywords_embedding']]

In [ ]:
X_numerical.shape, X_categorical.shape, X_text.shape, X_subtypes.shape, X_types.shape, X_keys.shape

((25489, 4), (25489, 15), (25489, 1), (25489, 1), (25489, 1), (25489, 1))

In [ ]:
X_text['remainder__text_embedding'][0].shape, X_subtypes['remainder__sub_types_embedding'][0].shape, X_types['remainder__types_embedding'][0].shape, X_keys['remainder__keywords_embedding'][0].shape

(TensorShape([1, 79, 768]),
 TensorShape([1, 15, 768]),
 TensorShape([1, 11, 768]),
 TensorShape([1, 19, 768]))

In [ ]:
# Concatenate padded arrays along axis 1
X_text_concatenated = np.concatenate(X_text_padded, axis=1)
X_subtypes_concatenated = np.concatenate(X_subtypes_padded, axis=1)
X_types_concatenated = np.concatenate(X_types_padded, axis=1)
X_keys_concatenated = np.concatenate(X_keys_padded, axis=1)

# Reshape to (-1, 768)
X_text = X_text_concatenated.reshape(-1, 768)
X_subtypes = X_subtypes_concatenated.reshape(-1, 768)
X_types = X_types_concatenated.reshape(-1, 768)
X_keys = X_keys_concatenated.reshape(-1, 768)

In [ ]:
X_text[0].shape, X_subtypes[0].shape, X_types[0].shape, X_keys[0].shape

(TensorShape([1, 79, 768]),
 TensorShape([1, 15, 768]),
 TensorShape([1, 11, 768]),
 TensorShape([1, 19, 768]))

In [ ]:
X_numerical_train, X_numerical_test, X_categorical_train, X_categorical_test, X_text_train, X_text_test,X_subtypes_train, X_subtypes_test, X_types_train, X_types_test, X_keys_train, X_keys_test, y_train, y_test = train_test_split(
    X_numerical, X_categorical, X_text, X_subtypes, X_types, X_keys, y, test_size=0.30, random_state=42
)

In [ ]:
print("X_text_train[0].shape:", X_text_train[0].shape)
print("X_text_test[0].shape:", X_text_test[0].shape)

print("X_subtypes_train[0].shape:", X_subtypes_train[0].shape)
print("X_subtypes_test[0].shape:", X_subtypes_test[0].shape)

print("X_types_train[0].shape:", X_types_train[0].shape)
print("X_types_test[0].shape:", X_types_test[0].shape)

print("X_keys_train[0].shape:", X_keys_train[0].shape)
print("X_keys_test[0].shape:", X_keys_test[0].shape)

X_text_train[0].shape: (1, 93, 768)
X_text_test[0].shape: (1, 106, 768)
X_subtypes_train[0].shape: (1, 13, 768)
X_subtypes_test[0].shape: (1, 14, 768)
X_types_train[0].shape: (1, 11, 768)
X_types_test[0].shape: (1, 13, 768)
X_keys_train[0].shape: (1, 12, 768)
X_keys_test[0].shape: (1, 13, 768)


In [ ]:
def initialize_model():

    model_numerical = Sequential([
        Input(shape=(4,), name='input_numerical'),
        Dense(32, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
        Dense(8, activation='relu')
    ], name='model_numerical')

    model_categorical = Sequential([
        Input(shape=(15,), name='input_categorical'),
        Dense(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dropout(0.2),
        Dense(16, activation='relu')
    ], name='model_categorical')

    model_text = Sequential([
        Input(shape=(79, 768), name='input_text'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01), return_sequences=True),
        LSTM(64, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(256, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(16, activation='relu'),
    ], name='model_text')


    model_type = Sequential([
        Input(shape=(11, 768), name='input_type'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
    ], name='model_type')


    model_subtype = Sequential([
        Input(shape=(15, 768), name='input_subtype'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_subtype')


    model_keys = Sequential([
        Input(shape=(19, 768), name='input_keys'),
        LSTM(128, activation='relu', kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dense(32, activation='relu')
    ], name='model_keys')


    # Concatenate the outputs of all Sequential models
    concatenated = Concatenate()([
        model_numerical.output,
        model_categorical.output,
        model_text.output,
        model_type.output,
        model_keys.output,
        model_subtype.output
    ])

    output = Dense(1, activation='linear', name='output')(concatenated)


    model = models.Model(inputs=[model_numerical.input, model_categorical.input, model_text.input, model_type.input, model_keys.input, model_subtype.input], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['mse', 'accuracy'])

    return model


In [ ]:
def get_history():
    es = EarlyStopping(patience=10, restore_best_weights=False)

    train_data = {
        'input_numerical': X_numerical_train,
        'input_categorical': X_categorical_train,
        'input_text': X_text_train,
        'input_type': X_types_train,
        'input_subtype': X_subtypes_train,
        'input_keys': X_keys_train
    }

    test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }

    history = model.fit(
        train_data,
        y_train,
        epochs=20,
        batch_size=16,
        validation_data=(test_data, y_test),
        callbacks=[es],
        verbose=1
    )

    return history


In [ ]:
model = initialize_model()

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_text (InputLayer)     [(None, 79, 768)]            0         []                            
                                                                                                  
 lstm_10 (LSTM)              (None, 79, 128)              459264    ['input_text[0][0]']          
                                                                                                  
 lstm_11 (LSTM)              (None, 64)                   49408     ['lstm_10[0][0]']             
                                                                                                  
 dropout_41 (Dropout)        (None, 64)                   0         ['lstm_11[0][0]']             
                                                                                            

In [ ]:
get_history()

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type tensorflow.python.framework.ops.EagerTensor).

In [ ]:
test_data = {
        'input_numerical': X_numerical_test,
        'input_categorical': X_categorical_test,
        'input_text': X_text_test,
        'input_type': X_types_test,
        'input_subtype': X_subtypes_test,
        'input_keys': X_keys_test
    }
model.evaluate(test_data, y_test)

239/239 [==============================] - 0s 2ms/step - loss: 1885.5562 - mse: 1881.6256 - accuracy: 2.6154e-04


[1885.55615234375, 1881.6256103515625, 0.00026154046645388007]

In [ ]:
no_prices_df = df[df['usd'].isna()]